In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("vk.csv", delimiter=',')

In [3]:
df.head()

,u,v,t,h
0,3248374,11431799,27,6.0
1,3429130,4795235,13,8.0
2,105512,8910840,74,2.0
3,7900466,9739979,36,9.0
4,7296485,9838285,52,2.0


## Число вершин

In [4]:
vertices = len(set([*df["u"].unique(), *df["v"].unique()]))
print(vertices)

3103653


## Число всех ребер

In [5]:
print(len(df["u"]))

14847753


In [6]:
print(vertices*(vertices-1))

9632658840756


## Число уникальных ребер

In [7]:
unique_edges = len(np.unique(df[['u', 'v', 't', 'h']].values))
# unique_edges = len(np.unique(df[['u', 'v']].values))
print(unique_edges)

3103730


## Плотность графа
Для неориентированного простого графа плотность графа с числом вершин V  определяется как отношение числа его рёбер E к числу рёбер полного графа

In [8]:
density = 2*unique_edges/((vertices*(vertices-1)))
print(f'Плотность: {density}')

Плотность: 6.444181303023101e-07


## Компоненты слабой связности

In [9]:
class Node:
    def __init__(self, key):
        self.key = key
        self.parent = self
        self.size = 1

class UnionFind(dict):
    def find(self, key):
        node = self.get(key, None)
        if node is None:
            node = self[key] = Node(key)
        else:
            while node.parent != node: 
                # walk up & perform path compression
                node.parent, node = node.parent.parent, node.parent
        return node

    def union(self, key_a, key_b):
        node_a = self.find(key_a)
        node_b = self.find(key_b)
        if node_a != node_b:  # disjoint? -> join!
            if node_a.size < node_b.size:
                node_a.parent = node_b
                node_b.size += node_a.size
            else:
                node_b.parent = node_a
                node_a.size += node_b.size

In [10]:
from collections import defaultdict

def find_components(line_iterator):
    forest = UnionFind()

    for line in line_iterator:
        forest.union(line[0], line[1])

    result = defaultdict(list)
    for key in forest.keys():
        root = forest.find(key)
        result[root.key].append(key)

    return list(result.values())

In [11]:
graph_edges = [edge for edge in df[["u","v"]].values]

In [12]:
graph_components = find_components(graph_edges)

In [14]:
print(f'Число компонент слабой связности: {len(graph_components)}')

max_component = max(graph_components, key=lambda i: len(i))
print(f'Число вершин в максимальной компоненте связности: {len(max_component)}')
print(f'Доля вершин в максимальной по мощности компоненте слабой связности: {len(max_component)/vertices}')

Число компонент слабой связности: 28175
Число вершин в максимальной компоненте связности: 3041820
Доля вершин в максимальной по мощности компоненте слабой связности: 0.9800773475643056
